# Module 3: Feature engineering

In this module, we cover

- The importance of data quality
- Sources of error
- Model interpretation
- Feature transformation and scaling
- Feature engineering
- Feature selection
- Handling imbalanced datasets
- Modelling subgroups
- Hands-on example of selecting features

The [notebooks](https://github.com/decisionmechanics/lt541v) for the course are available on GitHub. Clone or download them to follow along.

In this notebook, we make use of the following third-party packages.

```bash
pip install jupyterlab category-encoders imbalanced-learn numpy 'pandera[polars]' 'polars[all]' pyjanitor scipy seaborn xgbfir xgboost yellowbrick
```

Some of the packages we use have deprecation warnings. These can be disabled.

In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

## The importance of data quality

<img src="images/module3-gigo.png" alt="GIGO" width="500" />

ML is a data-driven tool. The quality of the data directly affects the performance of the model---garbage in, garbage out.

Data quality impacts

- **Model accuracy**: High-quality data enables the model to learn patterns accurately. Poor data quality, like missing values, noise, or biases, can lead to incorrect predictions and reduced model accuracy.

- **Generalisation:** Clean and representative data helps models generalise well to new, unseen data. Poor quality data can lead to overfitting, where the model performs well on training data, but poorly on new data.

- **Bias reduction**: If data is biased, the model will likely make biased decisions. Ensuring balanced and representative data reduces the chances of unfair outcomes, especially in areas like healthcare or hiring.

- **Interpretability and trust**: High-quality data improves interpretability, helping stakeholders understand and trust model decisions. When data is noisy or inconsistent, model outputs are harder to explain and justify.

- **Efficiency in training**: Good data quality reduces the need for complex preprocessing and error correction, speeding up model training and improving computational efficiency.

- **Cost savings**: Detecting and fixing data issues early on prevents costly adjustments in later stages. Poor data quality can lead to incorrect results, requiring re-training and re-validation.

Data goes through a lifecycle, and quality must be maintained at every stage.

<img src="images/module3-data-lifecycle.png" alt="Data lifecycle" width="200" />

Data governance processes help raise the quality bar.

<img src="images/module3-data-governance.png" alt="Data governance" width="200" />

What are some of the primary data sources in your organisation? How would you rate the quality of the data?

### Data formats

Data will rarely be provided in a nice, tabular CSV file.

We need to be able to convert to/from formats such as

- Excel
- JSON
- JSON lines
- Text
- Relational databases
- Non-relational databases

## Sources of errors

Quality problems can occur from many sources.

- **Data entry errors**: Human errors in manual data entry, such as typos or inconsistent formatting, can lead to incorrect values that skew model training.

- **Incomplete data**: Missing values, which may occur due to data collection limitations or human error, can reduce the information available to the model and affect its performance.

- **Outliers and noise**: Outliers, or values that deviate significantly from the norm, may be genuine but can also represent errors or anomalies in data. Noise, or random variations in data, can make it hard for the model to identify real patterns.

- **Sampling bias**: If a dataset does not represent the underlying population accurately, the model will learn biases that do not generalise well. For instance, if a dataset has a demographic imbalance, predictions may be biased toward the over-represented group.

- **Data drift**: Over time, data may change in its properties or distribution, especially in dynamic fields like finance or social media. Models trained on outdated data may perform poorly on new data.

- **Measurement and instrumentation errors**: Errors in the tools or processes used to collect data, such as faulty sensors or inaccurate measurements, introduce inaccuracies that can lead to incorrect model predictions.

- **Data duplication**: Duplicated data points can lead to over-representation of certain patterns, which can bias model training and result in overfitting.

- **Labelling errors**: In supervised learning, incorrect labels can mislead the model during training. These errors often arise in tasks that require subjective judgment, such as image labelling or sentiment analysis.

- **Inconsistent data formats**: In datasets that integrate information from multiple sources, inconsistent formats or units (like mixing metric and imperial measurements) can introduce errors.

- **Human bias**: If data is collected in a way that reflects human biases, models trained on it may learn those biases. For example, biased language in text data can result in biased natural language models.

What other sources of errors have you encountered?

### Data validation

We can formally validate our data according to a schema. Specifications such as [JSON Schema](https://json-schema.org) allow us to exert greater control over the quality of our datasets.

[Pandera](https://pandera.readthedocs.io/en/stable/) is a library that allows us to perform data validation on dataframes, including Polars dataframes. Using Pandera, we can validate

- column names
- field types
- non-nullablity
- numeric limits (e.g. > 0)
- numeric ranges
- valid categorical values
- value patterns (via regex)
- column-wide statistics (e.g. median, standard deviation)
- custom validation rules

Let's create a simple schema for (partially) validating the shark incident dataset.

In [ ]:
import pandera.polars as pa


class SharkIncidentSchema(pa.DataFrameModel):
    uin: int
    incident_year: int = pa.Field(in_range={"min_value": 1700, "max_value": 2024})
    victim_injury: str = pa.Field(isin=["fatal", "injured", "uninjured"])
    latitude: float
    shark_length_m: float = pa.Field(gt=0, nullable=True)

We can now using this schema to validate our data at ingress.

In [ ]:
import polars as pl

try:
    (
        pl.scan_csv("data/shark-incidents.csv", infer_schema_length=None)
        .pipe(lambda df_: SharkIncidentSchema.validate(df_, lazy=True))
        .collect()
    )
except pa.errors.SchemaErrors as e:
    print(json.dumps(e.message, indent=2))

Once we fix the problems with the data, our data will validate and be read in successfully.

In [ ]:
import janitor.polars

(
    pl.read_csv("data/shark-incidents.csv", infer_schema_length=None)
    .clean_names()
    .filter(
        pl.col("uin").is_not_null(),
    )
    .with_columns(
        pl.col("victim_injury").replace(
            {
                "Injured": "injured",
            }
        ),
        pl.col("latitude").str.strip_chars().cast(pl.Float64),
    )
    .pipe(lambda df_: SharkIncidentSchema.validate(df_, lazy=True))
    .head()
)

Creating a schema for your dataset can be considered part of the EDA process. It forces you to be explicit about the nature of your data. The schema provides a formal description of what assumptions you are making about the incoming data.

And, if the assumptions are broken, it alerts you as the data drifts away from your expectations.

We can produce a more complete schema for validating the World Bank GDP dataset.

In [ ]:
class GdpSchema(pa.DataFrameModel):
    country_name: str = pa.Field(nullable=False)
    country_code: str = pa.Field(str_length=3, nullable=False)
    indicator_name: str = pa.Field(str_matches=r"GDP \(current US\$\)", nullable=False)
    indicator_code: str = pa.Field(str_matches=r"NY\.GDP\.MKTP\.CD", nullable=False)
    year: int = pa.Field(in_range={"min_value": 1960, "max_value": 2023})
    gdp: float = pa.Field(gt=0)

Is there anything missing from this schema? Could we make it tighter?

We can now validate the dataset as we injest it.

In [ ]:
(
    pl.read_csv(
        "data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv",
        skip_rows=4,
    )
    .clean_names()
    .unpivot(
        index=["country_name", "country_code", "indicator_name", "indicator_code"],
        variable_name="year",
        value_name="gdp",
    )
    .with_columns(
        pl.col("year").replace("", None),
        pl.col("gdp").replace("", None),
    )
    .with_columns(
        pl.col("year").cast(pl.Int64),
        pl.col("gdp").cast(pl.Float64),
    )
    .filter(pl.col("gdp").is_not_null())
    .pipe(lambda df_: GdpSchema.validate(df_, lazy=True))
    .head()
)

Take a dataset---one of your own, or one from the course---and define a set of constraints that should be applied to ensure it's valid.

## Model interpretation

White box models, such as logistic regression and decision trees, allow us to peer inside the model. We can review how it is using the features in making its predictions. This can give us insights that we can use to tweak the behaviour.

We'll use data from Kaggle's [2018 Machine Learning and Data Science Survey](https://www.kaggle.com/datasets/kaggle/kaggle-survey-2018) to see what insights the models can provide.

The data will be used to try and predict whether someone is a data scientist or a software engineer.

Build a transformation pipeline to prepare the data.

In [ ]:
from feature_engine import encoding, imputation
from sklearn import pipeline
from sklearn.compose import ColumnTransformer

categorical_pipeline = pipeline.Pipeline(
    [
        (
            "encode_categoricals",
            encoding.OneHotEncoder(
                top_categories=5,
                drop_last=True,
            ),
        )
    ]
)

numeric_pipeline = pipeline.Pipeline(
    [
        (
            "imputate",
            imputation.MeanMedianImputer(imputation_method="median"),
        ),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("categorical_pipeline", categorical_pipeline, ["gender", "country", "major"]),
        (
            "numeric_pipeline",
            numeric_pipeline,
            ["age", "education", "experience", "compensation"],
        ),
    ],
    remainder="passthrough",
)

column_transformer.set_output(transform="polars")

data_preperation_pipeline = pipeline.Pipeline(
    [
        ("column_transformer", column_transformer),
    ]
)

Load the data and split it into features and targets.

In [ ]:
import polars as pl

survey_df = pl.read_parquet("data/ml-ds-survey.parquet")

X_df = survey_df.select(
    [
        "gender",
        "country",
        "age",
        "education",
        "major",
        "experience",
        "compensation",
        "python",
        "r",
        "sql",
    ]
)

y_df = survey_df.select("title")

Split the data into training and test datasets.

In [ ]:
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

SEED = 123

X_train_df, X_test_df, y_train_df_, y_test_df_ = model_selection.train_test_split(
    X_df, y_df, test_size=0.3, random_state=SEED, stratify=y_df
)

X_train_df_ = data_preperation_pipeline.fit_transform(X_train_df.to_pandas())
X_test_df_ = data_preperation_pipeline.fit_transform(X_test_df.to_pandas())
y_train = LabelEncoder().fit_transform(y_train_df_.get_column("title"))
y_test = LabelEncoder().fit_transform(y_test_df_.get_column("title"))

This data can now be used to train a variety of models.

### Logistic regression interpretation

Logistic regression, like linear regression, finds coefficients for each feature. A weighted sum of the features is then used to make the prediction.

Build a job title classifier using logistic regression.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

logistic_regression_classifier = LogisticRegression(penalty=None)
logistic_regression_classifier.fit(scaler.fit_transform(X_train_df_), y_train)

logistic_regression_classifier.score(scaler.transform(X_test_df_), y_test)

How did the model weight the features?

In [ ]:
logistic_regression_classifier.coef_

It's easier to view this graphically.

In [ ]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": X_train_df_.columns,
            "coefficient": logistic_regression_classifier.coef_[0],
        }
    )
    .with_columns(pl.col("feature").str.split("__").list.last())
    .sort("coefficient", descending=True)
    .plot.bar(
        x="coefficient",
        y=alt.Y("feature", sort=None),
    )
    .properties(
        width=800,
        height=500,
    )
)

Features with positive coefficients offer more support for a prediction of software engineer. Those with negative coefficents push more to a prediction of data scientist.

Years of experience and majoring in Computer Science are strong indicators that you are a software engineer. Data scientist is a relatively recent career path.

R experience and level of education are strong predictors that you are a data scientist. Data scientists tend to need a broad range of technical skills.

Gender doesn't seem to play a particularly significant role. Maybe we could remove it from the model.

In [ ]:
logistic_regression_classifier = LogisticRegression(penalty=None)
logistic_regression_classifier.fit(
    scaler.fit_transform(X_train_df_.select(pl.exclude("^.*gender.*$"))), y_train
)

logistic_regression_classifier.score(
    scaler.transform(X_test_df_.select(pl.exclude("^.*gender.*$"))), y_test
)

This results in a slightly less accurate, but simpler, model.

We can review the new coefficients.

In [ ]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": X_train_df_.select(pl.exclude("^.*gender.*$")).columns,
            "coefficient": logistic_regression_classifier.coef_[0],
        }
    )
    .with_columns(pl.col("feature").str.split("__").list.last())
    .sort("coefficient", descending=True)
    .plot.bar(
        x="coefficient",
        y=alt.Y("feature", sort=None),
    )
    .properties(
        width=600,
        height=400,
    )
)

### Decision tree interpretation

Decision trees are also white-box models---we can review how they are making their predictions. Let's apply a decision tree classifier to our job title problem.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_classifier = DecisionTreeClassifier(max_depth=4, random_state=SEED)
decision_tree_classifier.fit(scaler.fit_transform(X_train_df_), y_train)

decision_tree_classifier.score(scaler.transform(X_test_df_), y_test)

We can see what features the decision tree classifier considered to be important.

In [ ]:
decision_tree_classifier.feature_importances_

Decision trees don't need to consider all the features. Some features may not be used at all in the partitioning process.

This means that decision trees perform automatic feature selection.

Again, it's easier to interpret the feature importances if we plot them.

In [ ]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": X_train_df_.columns,
            "importance": decision_tree_classifier.feature_importances_,
        }
    )
    .filter(
        pl.col("importance") > 0,
    )
    .with_columns(pl.col("feature").str.split("__").list.last())
    .sort("importance", descending=True)
    .plot.bar(
        x="importance",
        y=alt.Y("feature", sort=None),
    )
    .properties(
        width=600,
        height=400,
    )
)

We don't have +ve and -ve feature importances, as with logistic regression coefficients, as decision trees can fit non-linear relationships. This means that features can support _both_ predictions in different parts of the tree.

Decision trees may pick out different features that logistic regression, but the fact we see some correlation between the two models gives us some confidence that features such as R experience, college major, level of education and years of experience are probably useful indicators.

We can visualise part of the tree.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

title_class_names = sorted(y_df["title"].unique())

plt.figure(figsize=(10, 10), dpi=300)
plot_tree(
    decision_tree_classifier,
    feature_names=[column_name.split("__")[-1] for column_name in X_train_df_.columns],
    class_names=title_class_names,
    filled=True,
    max_depth=1,
)
plt.show()

This aligns with the feature importances.

### Random forest interpretation

Random forests are _theorectically_ white-box models, but, in practice, they can be more challenging to interpret. Let's apply a random forest classifier to our job title problem.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(max_depth=4, random_state=SEED)
random_forest_classifier.fit(scaler.fit_transform(X_train_df_), y_train)

random_forest_classifier.score(scaler.transform(X_test_df_), y_test)

We can examine the model's learning curve to see if it's under or overfitting.

In [ ]:
from yellowbrick.model_selection import learning_curve

learning_curve(random_forest_classifier, X_train_df_.to_pandas(), y_train_df_["title"]);

How many trees are in our forest?

In [ ]:
len(random_forest_classifier.estimators_)

Which features are important in the random forest?

In [ ]:
random_forest_classifier.feature_importances_

It's considering _all_ the features, which is suprising given what we saw with the simpler models. This can be an indication that there's overfitting.

Let's plot the feature importances.

In [ ]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": X_train_df_.columns,
            "importance": random_forest_classifier.feature_importances_,
        }
    )
    .filter(
        pl.col("importance") > 0,
    )
    .with_columns(pl.col("feature").str.split("__").list.last())
    .sort("importance", descending=True)
    .plot.bar(
        x="importance",
        y=alt.Y("feature", sort=None),
    )
    .properties(
        width=600,
        height=400,
    )
)

Again, we are seeing that R experience, college major, education and experience are all important.

We can review individual trees from the forest.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

title_class_names = sorted(y_df["title"].unique())

plt.figure(figsize=(10, 10), dpi=300)
plot_tree(
    random_forest_classifier.estimators_[0],
    feature_names=[column_name.split("__")[-1] for column_name in X_train_df_.columns],
    class_names=title_class_names,
    filled=True,
    max_depth=1,
)
plt.show()

This is of limited value as the model is the ensemble of all 100 trees. Looking at a single tree only provides a small part of the overall picture.

### XGBoost interpretation

As with random forests, XGBoost models are _theorectically_ white-box models, but, in practice, they are challenging to interpret. The idea of fitting chains of residuals is challenging to map to an intuitive interpretation---even more so that with random forests.

Let's apply an XGBoost classifier to our job title problem.

In [ ]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier(max_depth=4, random_state=SEED)
xgb_classifier.fit(scaler.fit_transform(X_train_df_), y_train)

xgb_classifier.get_booster().feature_names = X_train_df_.columns

xgb_classifier.score(scaler.transform(X_test_df_), y_test)

The XGBoost model is less accurate than the random forest. Is there any indication that we are overfitting?

Let's review the learning curve.

In [ ]:
learning_curve(xgb_classifier, X_train_df_.to_pandas(), y_train);

Which features are important in the model?

In [ ]:
xgb_classifier.feature_importances_

It's considering most of the features. The only one it's ignoring is people who prefer to self-describe their gender.

Let's plot the feature importances.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
xgb.plot_importance(xgb_classifier, ax=ax);

Here we are seeing that experience and education are still prominent, but compensation and age have risen to the top.

We can review individual boosted trees.

In [ ]:
fig, ax = plt.subplots(figsize=(48, 48))
xgb.plot_tree(xgb_classifier, num_trees=0, rankdir="LR", ax=ax);

Another approach we can use to try and interpret an XGBoost model is to train a decision tree on it's predictions and then review the decision tree.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

surrogate_regressor = DecisionTreeRegressor(max_depth=4)
surrogate_regressor.fit(
    X_train_df_, xgb_classifier.predict_proba(scaler.fit_transform(X_train_df_))
)

In [ ]:
import altair as alt

(
    pl.DataFrame(
        {
            "feature": X_train_df_.columns,
            "importance": surrogate_regressor.feature_importances_,
        }
    )
    .filter(
        pl.col("importance") > 0,
    )
    .with_columns(pl.col("feature").str.split("__").list.last())
    .sort("importance", descending=True)
    .plot.bar(
        x="importance",
        y=alt.Y("feature", sort=None),
    )
    .properties(
        width=600,
        height=400,
    )
)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

title_class_names = sorted(y_df["title"].unique())

plt.figure(figsize=(10, 10), dpi=300)
plot_tree(
    surrogate_regressor,
    feature_names=[column_name.split("__")[-1] for column_name in X_train_df_.columns],
    class_names=title_class_names,
    filled=True,
    max_depth=1,
)
plt.show()

## Feature transformation and scaling

We can transform and scale individual features to improve performance, accuracy and, occasionally, interpretation.

Popular approaches include

- Z-scores
- MinMax scaling
- MaxAbs scaling
- Robust scaling
- Quantile transformer scaling
- Power transformer scaling
- Unit vector normalisation
- Log transformation

### Z-scores

Calculating z-scores is a popular scaling method. Z-scores have a mean of 0 and a standard deviation of 1.

Centering the data can be useful when interpreting the intercept in linear regression, for example. Scaling to unit standard deviation can also reduce the impact of outliers.

In [ ]:
(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(
        (pl.col("shark_length_m") - pl.col("shark_length_m").mean())
        / pl.col("shark_length_m").std()
    )
    .filter(pl.col("shark_length_m").is_not_null())
    .head()
)

### MinMax scaling

MinMax scaling is popular as it's easy to explain. It's also useful if you have an algorithm that requires a specific range of values (such as 0-1) or if you want to preserve the original shape of the distribution.

In [ ]:
(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(
        (pl.col("shark_length_m") - pl.col("shark_length_m").min())
        / (pl.col("shark_length_m").max() - pl.col("shark_length_m").min())
    )
    .filter(pl.col("shark_length_m").is_not_null())
    .head()
)

### MaxAbs scaling

MaxAbs scaling scales the data into the range [-1, 1]. It's computationally efficient and is appropriate if you wish to preserve the signs of the values. 

In [ ]:
(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(
        pl.col("shark_length_m") / pl.col("shark_length_m").max().abs(),
    )
    .filter(pl.col("shark_length_m").is_not_null())
    .head()
)

### Robust scaling

As robust scaling makes use of the IQR, it's robust to outliers.

In [ ]:
(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(
        (pl.col("shark_length_m") - pl.col("shark_length_m").quantile(0.25))
        / (
            pl.col("shark_length_m").quantile(0.75)
            - pl.col("shark_length_m").quantile(0.25)
        )
    )
    .filter(pl.col("shark_length_m").is_not_null())
    .head()
)

### Quantile transformer scaling

Quantile scaling maps the original data to follow a specific distribution, typically a uniform or normal distribution. It is often used to reduce the impact of outliers and make data distributions more comparable by transforming data to match a chosen reference distribution.

Examine the distribution of the distance from shore at which the incident occurred.

In [ ]:
(
    pl.read_parquet("data/shark-incidents.parquet")
    .filter(pl.col("distance_to_shore_m").is_not_nan())
    .get_column("distance_to_shore_m")
    .hist(bin_count=30)
    .plot.bar(x="breakpoint", y="count")
)

Compare that with the data after quantile scaling.

In [ ]:
import polars.selectors as cs
from sklearn.preprocessing import QuantileTransformer

quantile_transformer = QuantileTransformer()

(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(cs.numeric().exclude((["latitude", "longitude"])))
    .pipe(
        lambda df_: pl.from_numpy(
            quantile_transformer.fit_transform(df_),
            schema=df_.columns,
        )
    )
    .filter(pl.col("distance_to_shore_m").is_not_nan())
    .get_column("distance_to_shore_m")
    .hist(bin_count=30)
    .plot.bar(x="breakpoint", y="count")
)

### Log transformation

Log transformations allow us to convert highly skewed distributions to more symmetric distributions.

For example, financial data is usually highly right-skewed.

In [ ]:
gdp_per_capita = (
    pl.read_csv("data/world-bank-gdp.csv")
    .filter(
        pl.col("year") == 2023,
        pl.col("gdp").is_not_null(),
    )
    .with_columns(
        (pl.col("gdp") / pl.col("population")).alias("gdp_per_capita"),
    )
    .get_column("gdp_per_capita")
)

In [ ]:
(
    gdp_per_capita.hist(bin_count=15).plot.bar(
        x="breakpoint",
        y="count",
    )
)

If we take the log of GDP/capita, we see a more symmetric distribution.

In [ ]:
(
    gdp_per_capita.log()
    .hist(bin_count=15)
    .plot.bar(
        x="breakpoint",
        y="count",
    )
)

This makes it easier to see patterns and can improve the performance of machine learning algorithms. The downside is that it's harder to interpret the numbers in the context of the original domain.

### Power transformer scaling

Similarly to log transformations, power scaling is used to make data more normally distributed by applying a power-based transformation. This can help reduce skewness and stabilise variance in features, improving model performance for algorithms that are sensitive to the distribution of data.

It's more flexible than simple log scaling.

There are two common types of power transformations.

- Box-Cox (only applies to positive values)
- Yeo-Johnson (allows both positive and negative values)            

In [ ]:
from sklearn.preprocessing import PowerTransformer

power_transformer = PowerTransformer()

(
    pl.read_parquet("data/shark-incidents.parquet")
    .select(cs.float().exclude((["latitude", "longitude"])))
    .pipe(
        lambda df_: pl.from_numpy(
            power_transformer.fit_transform(df_),
            schema=df_.columns,
        )
    )
    .head()
)

### Unit vector normalisation

Unit vector normalization converts a vector into a unit vector with a magnitude of 1 while preserving its direction. This is done by dividing each component of the vector by its magnitude, calculated as the square root of the sum of its components squared.

This scaling approach is commonly using in algorithms that are sensitive to magnitude differences, such as k-NN or cosine similarity-based models (e.g. recommendation engines).

In [ ]:
from sklearn.preprocessing import Normalizer

normaliser = Normalizer()

normalized_df = (
    pl.read_parquet("data/shark-incidents.parquet")
    .select(cs.float().exclude((["latitude", "longitude"])))
    .filter(~pl.any_horizontal(pl.all().is_null()))
    .pipe(
        lambda df_: pl.from_numpy(
            normaliser.fit_transform(df_),
            schema=df_.columns,
        )
    )
    .head()
)

normalized_df

In [ ]:
(normalized_df.select(pl.all() ** 2).sum_horizontal().alias("sum_of_squares"))

## Feature engineering

Feature transformation and scaling are forms of feature engineering---i.e. the process of creating transforming, or extracting features from raw data to improve model performance.

We engineer features to enhance their predictive power.

### Dimensionality reduction

Principal Component Analysis (PCA) is a popular dimensionality reduction approach.

We'll use it here to reduce the dimensionality of the [Palmer penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

PCA can’t handle missing values, so we’ll just remove observations with missing data.

Probabilistic PCA (PPCA) can be used when you have missing data.

In [ ]:
penguin_df = pl.read_csv("data/penguins.csv").drop_nulls()

penguin_df

PCA only works with numeric values.

In [ ]:
penguin_numeric_df = penguin_df.select(cs.numeric().exclude("year"))

penguin_numeric_df

PCA is affected by scale, so we'll convert our dataset to z-scores.

In [ ]:
from scipy.stats import zscore

penguin_z_score_df = pl.from_numpy(zscore(penguin_numeric_df.to_numpy(), ddof=1)).pipe(
    lambda df_: df_.rename(dict(zip(df_.columns, penguin_numeric_df.columns)))
)

penguin_z_score_df

Transform the dataset to its principal components.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

principal_components = pca.fit_transform(penguin_z_score_df.to_numpy())

principal_component_df = pl.concat(
    [
        penguin_df.select("species"),
        pl.from_numpy(
            principal_components,
            ["component1", "component2", "component3", "component4"],
        ),
    ],
    how="horizontal",
)

principal_component_df

How much of the variable is explained by each principal component?

In [ ]:
import numpy as np

np.cumsum(pca.explained_variance_ratio_)

We can see that 88% of the total variance is explained by the first two components.

This lets us visualise the data as a two-dimensional scatterplot.

In [ ]:
(
    principal_component_df.plot.point(
        x="component1", y="component2", color="species"
    ).properties(width=500, height=500)
)

We can see that the Gentoos can be trivially identified, but seperating the Adelies from the Chinstraps is more challenging.

PCA isn't the only dimensionality reduction technique. There are alternative approaches.

One that is popular in classification problems is Linear Discriminant Analysis (LDA). LDA is a supervised learning approach that takes the class labels into account when looking to isolate components with maximum variance.

PCA focuses on capturing the most significant features in the data overall. LDA explicitly aims to improve the distinction between categories.

Again, we'll transform the penguin dataset.

In [ ]:
(pl.read_csv("data/penguins.csv").drop(["island", "sex", "year"]).drop_nulls())

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

penguin_df = (
    pl.read_csv("data/penguins.csv").drop(["island", "sex", "year"]).drop_nulls()
)

lda_df = pl.from_numpy(
    LinearDiscriminantAnalysis(n_components=2).fit_transform(
        penguin_df.drop("species"), penguin_df.select("species")
    ),
    schema=["component1", "component2"],
).with_columns(penguin_df.get_column("species"))

In [ ]:
(
    lda_df.plot.point(x="component1", y="component2", color="species").properties(
        width=500, height=500
    )
)

Compare this with the plot produced earlier using PCA. There appears to be better separation of the Adelies and Chinstraps.

[Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD) is another popular dimensionality reduction technique. It tends to be used in textual analysis and recommandation systems. It's more computationally intensive then PCA.

### Working with missing data

Many popular ML techniques can't handle datasets with missing values. Even [those that can](https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values) will often handle missing values in ways that are unintuitive, thus complicating interpretation.

The best approach to dealing with missing values is not to have any---i.e. improve the quality of your dataset. Of course, that's not always possible, but it's the first thing that should be considered. Are they missing values _really_ necessary, or is it a flaw in the data collection process.

The most common approach to dealing with missing values is to delete any observations where the data isn't complete. This is dangerous. Unless you are sure that missing values are randomly distributed within your dataset, this approach can bias your model.

It's rare to find that missing values don't have a _systemic_ cause. Reasons for missing data include

- Certain groups in a population may be less likely to respond
- A piece of equipment may be broken
- A data entry clerk may be untrained
- Someone may have performed an incorrect merge
- Data may not be available for certain days/times
- Network or storage failures
- Anonymisation

Can you think of any other reasons why data might be missing? Think about some of the dataset you have been working with recently.

So...if we can't avoid the missing data, and we can't remove observations without complete data, what _can_ we do?

We have to replace the missing values with our best estimate of what the actual value might be. We need to impute values for those that are missing.

Univariate imputation considers each variable _in isolation_ when considering how to impute missing values. This makes it easy to perform, but can result in values that don't make sense in the context of the entire observation.

We can impute missing shark lengths in our shark incident dataset.

In [ ]:
shark_lengths = pl.read_parquet("data/shark-incidents.parquet").get_column(
    "shark_length_m"
)

In [ ]:
shark_lengths.null_count()

Impute a literal value (0).

In [ ]:
(
    shark_lengths.fill_null(0)
    .value_counts()
    .plot.bar(
        x="shark_length_m",
        y="count",
    )
)

Impute the maximum value.

In [ ]:
(
    shark_lengths.fill_null(shark_lengths.max())
    .value_counts()
    .plot.bar(
        x="shark_length_m",
        y="count",
    )
)

Impute the mean value.

In [ ]:
(
    shark_lengths.fill_null(shark_lengths.mean())
    .value_counts()
    .plot.bar(
        x="shark_length_m",
        y="count",
    )
)

Impute the median value.

In [ ]:
(
    shark_lengths.fill_null(shark_lengths.median())
    .value_counts()
    .plot.bar(
        x="shark_length_m",
        y="count",
    )
)

None of these seem particularly satisfactory, but they at least let the ML model use all the data.

Multivariate feature imputation uses the non-missing values in an observation to predict (estimate) the missing values. Linear regression is a popular way of doing the prediction.

In [ ]:
gdp_2023_df = (
    pl.read_csv("data/world-bank-gdp.csv")
    .filter(pl.col("year") == 2023, pl.col("income_group") != "")
    .drop("year")
    .with_columns(pl.col("gdp").is_null().alias("missing_gdp"))
)

In [ ]:
gdp_2023_df.null_count()

In [ ]:
gdp_feature_df = gdp_2023_df.select(["income_group", "population", "gdp"]).to_dummies(
    "income_group"
)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer(max_iter=10, random_state=SEED)

iterative_imputed_value_df = pl.from_numpy(
    iterative_imputer.fit_transform(gdp_feature_df), schema=gdp_feature_df.columns
)

with pl.Config(tbl_rows=len(gdp_2023_df)):
    with pl.StringCache():
        pl.Series(
            ["Low income", "Lower middle income", "Upper middle income", "High income"]
        ).cast(pl.Categorical)

        print(
            gdp_2023_df.with_columns(
                pl.col("income_group").cast(pl.Categorical),
                iterative_imputed_value_df.get_column("gdp"),
            )
            .filter(pl.col("missing_gdp"))
            .select(["country_name", "income_group", "population", "gdp"])
            .sort(["income_group", "population"])
        )

Nearest neighbours imputation is another multivariate imputation approach.

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer()

knn_imputed_value_df = pl.from_numpy(
    knn_imputer.fit_transform(gdp_feature_df), schema=gdp_feature_df.columns
)

with pl.Config(tbl_rows=len(gdp_2023_df)):
    with pl.StringCache():
        pl.Series(
            ["Low income", "Lower middle income", "Upper middle income", "High income"]
        ).cast(pl.Categorical)

        print(
            gdp_2023_df.with_columns(
                pl.col("income_group").cast(pl.Categorical),
                knn_imputed_value_df.get_column("gdp"),
            )
            .filter(pl.col("missing_gdp"))
            .select(["country_name", "income_group", "population", "gdp"])
            .sort(["income_group", "population"])
        )

### Encoding categorical variables

Categorical variables (such as education level, colours, etc.) need to be encoded before they can be used as features in an ML model.

If the feature only has a few values, we've seen that we can transform them into dummy variables. A new feature (column) is created for each level (value), with a 1 indicating the presence of that value, and 0 its absence.

We generally denote one value as the reference value. That is the value that is indicated by the absence of all the others. So, if we had a categorical feature called `day_of_week` we might have `Sunday` as the reference value, and the dummy features would be `day_of_week_monday` to `day_of_week_saturday`. There would be no column for the reference value as it would be the default if all the other day features were 0---i.e. if it's not Monday, Tuesday, Wednesday, Thursday, Friday or Saturday then it must be Sunday.
                                                                                                                                                                
In general, if we have $n$ values, we will end up with $n-1$ dummy features.                                                                                       

In [ ]:
(
    gdp_2023_df.with_columns(pl.col("income_group").alias("income_group_"))
    .select(["income_group", "income_group_"])
    .to_dummies("income_group_", drop_first=True)
)

If the categorical feature has many unique values, we can use binary encoding. This will constrain the number of dummy features to be $log_{2}$ of the number of unique values.

In [ ]:
from category_encoders import BinaryEncoder

gdp_df = pl.read_csv("data/world-bank-gdp.csv")

binary_encodings = pl.from_pandas(
    BinaryEncoder(cols="country_code").fit_transform(
        gdp_df.select(["country_name", "country_code"]).to_pandas()
    )
)

binary_encodings.sample(20)

If there's a natural order to the data, we can encode that.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encodings = OrdinalEncoder(
    categories=[
        ["Low income", "Lower middle income", "Upper middle income", "High income"]
    ]
).fit_transform(
    gdp_df.filter(pl.col("income_group") != "").select(pl.col("income_group"))
)

(
    gdp_df.filter(pl.col("income_group") != "")
    .select(pl.col("income_group"))
    .with_columns(pl.Series(name="encoding", values=ordinal_encodings))
)

Target encoding converts categorical variables into numerical values based on the relationship between each category and the target variable.

In [ ]:
from sklearn.preprocessing import TargetEncoder

shark_incident_df = pl.read_parquet("data/shark-incidents.parquet")

pl.from_numpy(
    TargetEncoder().fit_transform(
        shark_incident_df.select("shark_common_name"),
        shark_incident_df.get_column("victim_injury"),
    )
).with_columns(
    shark_incident_df.select("shark_common_name"),
)

As this is an ML technique, we may see variations due to fold splits, etc.

### Extracting temporal features from timestamps

In [ ]:
(
    shark_incident_df.filter(pl.col("time_of_incident").is_not_null())
    .select("time_of_incident")
    .sample(20)
)

In [ ]:
(
    shark_incident_df.with_columns(
        shark_incident_df.get_column("time_of_incident")
        .cut(breaks=[500, 800, 1200, 1500, 1700, 1900, 2100], left_closed=True)
        .replace_strict(
            {
                "[-inf, 500)": "night",
                "[500, 800)": "early morning",
                "[800, 1200)": "late morning",
                "[1200, 1500)": "early afternoon",
                "[1500, 1700)": "late afternoon",
                "[1700, 1900)": "early evening",
                "[1900, 2100)": "late evening",
                "[2100, inf)": "night",
            }
        )
        .alias("period_of_incident")
    )
    .select(["time_of_incident", "period_of_incident"])
    .sample(20)
)

In [ ]:
(
    shark_incident_df.with_columns(
        pl.col("time_of_incident")
        .cut(breaks=[500, 800, 1200, 1500, 1700, 1900, 2100], left_closed=True)
        .replace_strict(
            {
                "[-inf, 500)": "night",
                "[500, 800)": "early morning",
                "[800, 1200)": "late morning",
                "[1200, 1500)": "early afternoon",
                "[1500, 1700)": "late afternoon",
                "[1700, 1900)": "early evening",
                "[1900, 2100)": "late evening",
                "[2100, inf)": "night",
            }
        )
        .alias("period_of_incident")
    )
    .select(["time_of_incident", "period_of_incident"])
    .filter(pl.col("time_of_incident").is_not_null())
    .unique()
    .sort("time_of_incident")
)

### Interaction terms

Interaction effects in machine learning occur when the relationship between one feature (or variable) and the target outcome depends on the value of another feature. For example, in a model predicting house prices, the effect of a house's size on price might depend on its location—larger houses may increase in value more significantly in urban areas than in rural ones.

In linear modelling, we can represent this as follows.

$$
y = \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + \beta_{3}x_{1}x_{2} + \epsilon
$$

We can then use $\beta_{3}$ to determine the _type_ of interaction effect.

- If $\beta_{3}$ is positive, and $x_{1}$ and $x_{2}$ also have an impact, then the interaction is synergystic.
- If $\beta_{3}$ is negative, and $x_{1}$ and $x_{2}$ also have an impact, then the interaction is antagonistic.
- If $\beta_{3}$ is negligible, then the relationship between $x_{1}$ and $x_{2}$ is additive.
- If $\beta_{3}$ is non-negligible, then it's rare that either $x_{1}$ or $x_{2}$ wouldn't also have an impact. This would be atypical.

We can create a new interaction term by multiplying columns together.

In [ ]:
(
    survey_df.with_columns(
        (pl.col("education").cast(pl.Int64) * pl.col("experience")).alias(
            "education_experience"
        )
    )
)

Approaches such as decision trees and neural networks can model interaction effects directly. Other techniques, such as logistic regression, require the interaction terms to be engineered.

In the case of decision trees, a feature, $X_{1}$, does _not_ interact with another feature, $X_{2}$ if

- they are not on the same branch or
- $X_{2}$ is used the same way in each branch

The [XGBoost Feature Interactions Reshaped](https://github.com/limexp/xgbfir) library ranks features and feature interactions in XGBoost models.

It ranks the feature interactions according to various metrics, and provides an "Average Rank" score.

The library produces an Excel workbook, which we'll query using Polars.

Write the report to `temp/survey-fi.xlsx`.

In [ ]:
import xgbfir

xgbfir.saveXgbFI(
    xgb_classifier,
    feature_names=X_train_df_.columns,
    OutputXlsxFile="temp/survey-fi.xlsx",
)

What worksheets are in the workbook?

In [ ]:
from openpyxl import load_workbook

workbook = load_workbook("temp/survey-fi.xlsx", read_only=True)
workbook.sheetnames

The "Depth 0" worksheet contains the feature metrics. The "Depth 1" worksheet shows interactions between _pairs_ of features. And, as you might have guessed, the "Depth 2" worksheet contains interactions between _three_ metrics.

Review the features, ranked by average rank.

In [ ]:
(
    pl.read_excel("temp/survey-fi.xlsx", sheet_name="Interaction Depth 0")
    .with_columns(
        pl.col("Interaction")
        .str.replace_all("numeric_pipeline__", "")
        .str.replace_all("categorical_pipeline__", "")
    )
    .sort("Average Rank")
)

R experience, education and years of experience continue to be highlighted.

Gender seems to be less important than other features.

What about the interactions between pairs of features. Review the highest ranked interactions.

In [ ]:
with pl.Config(tbl_rows=1000, fmt_str_lengths=1000, tbl_width_chars=1000):
    print(
        pl.read_excel("temp/survey-fi.xlsx", sheet_name="Interaction Depth 1")
        .with_columns(
            pl.col("Interaction")
            .str.replace_all("numeric_pipeline__", "")
            .str.replace_all("categorical_pipeline__", "")
        )
        .sort("Average Rank")
        .select(["Interaction", "Average Rank"])
        .head(10)
    )

Education tends to be followed by experience.

Let's see if these values are correlated.

In [ ]:
survey_df.select(pl.corr("education", "experience", method="spearman"))

It's possible that the model is teasing out a non-linear relationship.

In [ ]:
import seaborn.objects as so

(
    so.Plot(survey_df, x="experience", y="education", color="title")
    .add(so.Dots(alpha=0.9, pointsize=2), so.Jitter(x=0.7, y=1))
    .add(so.Line(), so.PolyFit())
    .scale(color="viridis")
)

Mid-career data scientists appear to have more education that those at earlier or later stages of their careers. This effect isn't as pronounced for software engineers.

What about a relationship between R experience and compensation?

In [ ]:
(
    survey_df.select(["compensation", "r"])
    .sample(1000)
    .plot.boxplot(x="r", y="compensation")
)

We can also review the interactions between three features.

In [ ]:
with pl.Config(tbl_rows=1000, fmt_str_lengths=1000, tbl_width_chars=1000):
    print(
        pl.read_excel("temp/survey-fi.xlsx", sheet_name="Interaction Depth 2")
        .with_columns(
            pl.col("Interaction")
            .str.replace_all("numeric_pipeline__", "")
            .str.replace_all("categorical_pipeline__", "")
        )
        .sort("Average Rank")
        .select(["Interaction", "Average Rank"])
        .head(10)
    )

Note that gender doesn't seem to feature prominently in the interaction metrics, suggesting it doesn't have a direct _or_ synergystic/antagonistic effect.

## Feature selection

Feature selection is the process of selecting the most relevant features from the existing feature set by removing redundant, irrelevant or noisy ones.

The goal to is reduce dimensionality, prevent overfitting and improve performance.

### Domain expertise

Domain expertise can be the most significant tool in feature selection. Understanding how the variables are related to each other, and to the value to be predicted, can significantly simplify the modelling burden.

Influence diagrams are an effective way to represent domain knowledge. The capture the _causal_ relationships between variables, which are difficult to tease out of traditional machine learning models.

<img src="images/module3-influence-diagram.png" alt="Influence diagram" />

### Correlated features/targets

We can examine correlations between features, and between features and the target.

Correlated features may suggest double counting.

Features that are correlated with the target may be useful predictors.

In [ ]:
import seaborn as sns

sns.heatmap(
    pl.from_pandas(
        data_preperation_pipeline.fit_transform(
            X_df.with_columns(y_df)
            .with_columns(
                pl.when(pl.col("title") == "Software Engineer")
                .then(1)
                .otherwise(0)
                .alias("title")
            )
            .to_pandas()
        )
        .rename(
            lambda column_name: column_name.replace("categorical_pipeline__", "")
            .replace("numeric_pipeline__", "")
            .replace("remainder__", "")
        )
        .rename(
            {
                "major_Computer science (software engineering, etc.)": "major_cs",
            }
        )
        .to_pandas()
        .corr(method="spearman")
        .reset_index(names="variable")
    )
    .select(
        "variable",
        "age",
        "education",
        "experience",
        "compensation",
        "major_cs",
        "r",
        "title",
    )
    .to_pandas()
    .set_index("variable"),
    cmap="RdBu",
    annot=True,
    fmt=".1f",
    vmin=-1,
    vmax=1,
).set(xlabel="", ylabel="");

We can see that age and experience are quite strongly correlated. Age also seems quite weakly correlated with the target variable.

Country also seems to be correlated with compensation, meaning that country may just be acting as a proxy.

We could look at correlations between interaction terms and the target, although we quickly hit problems due to combinatorial explosion.

### Removing uninformative features

Features with low variance don't provide much predictive value. We may wish to remove them.

In [ ]:
scaler.transform(X_test_df_)

In [ ]:
(
    X_train_df_.with_columns(
        (pl.all() - pl.all().min()) / (pl.all().max() - pl.all().min())
    )
    .var()
    .transpose(include_header=True, header_name="feature", column_names=["variance"])
    .with_columns(
        pl.col("feature")
        .str.replace_all("categorical_pipeline__", "")
        .str.replace_all("numeric_pipeline__", "")
    )
    .sort("variance")
)

We need to balance the lack of information in the feature with the need to include data about minority groups.

### Statistical filtering

We can select features based on how they perform on statistical tests against the target column.

In [ ]:
from sklearn.feature_selection import SelectKBest

penguin_df = (
    pl.read_csv("data/penguins.csv")
    .select(
        [
            "body_mass_g",
            "flipper_length_mm",
            "bill_depth_mm",
            "bill_length_mm",
            "species",
        ]
    )
    .drop_nulls()
)

k_best_selector = SelectKBest(k=2).fit(
    penguin_df.drop("species"), penguin_df.select("species")
)

column_indexes = k_best_selector.get_support(indices=True)

penguin_df.select(pl.nth(column_indexes.tolist()))

### Sequential feature selection

Sequential feature selection iteratively adds (forward selection) or removes (backward selection) features. The choice of which feature to add/remove is determined by training models with each of the candidate features added/removed and scoring them. The feature that gives the best/worst result is added/removed.

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(
    LogisticRegression(), n_features_to_select=2, direction="forward"
)

sfs_fit = sfs.fit(penguin_df.drop("species"), penguin_df.select("species"))

In [ ]:
penguin_df.drop("species")[:, sfs_fit.support_.tolist()]

### Recursive feature elimination

Recursive feature elimination (RFE) is a wrapper selection method. It use feature importances to gradually remove features, refitting the model on the remaining features at each stage. A number of features are removed in each cycle.

In [ ]:
from sklearn.feature_selection import RFE

rfe = RFE(LogisticRegression(), n_features_to_select=2)

rfe_fit = rfe.fit(penguin_df.drop("species"), penguin_df.select("species"))

What two features were retained?

In [ ]:
penguin_df.drop("species")[:, rfe_fit.support_.tolist()]

How were the features ranked (selected ones have a rank of 1)?

In [ ]:
rfe_fit.ranking_

We can also perform RFE using cross-validation splits.

In [ ]:
from sklearn.feature_selection import RFECV

rfe_cv = RFECV(LogisticRegression())

rfe_cv_fit = rfe_cv.fit(penguin_df.drop("species"), penguin_df.select("species"))

In [ ]:
penguin_df.drop("species")[:, rfe_cv_fit.support_.tolist()]

### Tree-based feature selection

Tree-based algorithms perform their own feature selection. The are free to completely ignore unhelpful features when calculating the optimal splits.

In [ ]:
pl.DataFrame(
    {
        "feature": penguin_df.drop("species").columns,
        "importance": DecisionTreeClassifier(max_depth=2)
        .fit(penguin_df.drop("species"), penguin_df.select("species"))
        .feature_importances_,
    }
)

We can see that the decision tree didn't consider body mass to be a useful feature.

### Feature selection using grid search

We can combine RFE with grid search to search both the feature and hyperparameter space.

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline

rfe_gs_base_classifier = RandomForestClassifier(random_state=SEED)

rfe_gs = RFECV(rfe_gs_base_classifier, cv=StratifiedKFold(2))

rfe_gs.fit(scaler.fit_transform(X_train_df_), y_train)

pipeline = Pipeline(
    [("feature_selection", rfe_gs), ("classification", rfe_gs_base_classifier)]
)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid={
        "classification__max_depth": [2, 3, 4, 5],
    },
    cv=StratifiedKFold(2),
)

grid_search.fit(
    scaler.fit_transform(X_train_df_),
    y_train,
)

Which features were selected?

In [ ]:
X_train_df_[:, rfe_gs.support_.tolist()].columns

What were the optimal hyperparameters?

In [ ]:
grid_search.best_params_

How good is the optimal model?

In [ ]:
grid_search.best_score_

## Handling imbalanced datasets

As we have seen when discussing the accuracy metric, unbalanced target classes can cause problems with ML.

In addition to using more informative evaluation metrics (e.g. F1 score, AUC) we can resample our datasets to increase the occurrences of the minority class (oversampling) or reducing the occurences of the majority class (undersampling). We can also adjust the threshold probabilities to assist with classifying minority classes.

We can also use algorithms that are more effective at handling unbalanced classes.

### Resampling

To increase the instances of the minority classes, we can simply resample them, with replacement. We only need to resample to balance classes in the training data.

Other resampling techniques, such as Synthetic Minority Oversampling Technique (SMOTE), create synthetic observations, based on the existing data. SMOTE makes use of k-nearest neigbbours to generate its synthetic instances.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X_unbalanced, y_unbalanced = make_classification(
    n_classes=2,
    class_sep=2,
    weights=[0.1, 0.9],
    n_informative=3,
    n_redundant=1,
    flip_y=0,
    n_features=8,
    n_clusters_per_class=1,
    n_samples=1000,
    random_state=SEED,
)

X_train_unbalanced, X_test_unbalanced, y_train_unbalanced, y_test_unbalanced = (
    train_test_split(X_unbalanced, y_unbalanced, test_size=0.3, random_state=SEED)
)

In [ ]:
X_train_unbalanced.shape

In [ ]:
pl.Series(name="y", values=y_train_unbalanced).value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=SEED)

X_train_resampled, y_train_resampled = smote.fit_resample(
    X_train_unbalanced, y_train_unbalanced
)

In [ ]:
X_train_resampled.shape

In [ ]:
pl.Series(name="y", values=y_train_resampled).value_counts()

### Balanced bagging classifier

A balanced bagging classifier is a wrapper around a base classifier, such as a decision tree classifier, that incorporates target class balancing into the training phase.

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import classification_report

base_classifier = DecisionTreeClassifier(max_depth=4, random_state=SEED)

balanced_bagging_classifier = BalancedBaggingClassifier(
    base_classifier,
    sampling_strategy="auto",
    replacement=False,
    random_state=SEED,
)

balanced_bagging_classifier.fit(scaler.fit_transform(X_train_df_), y_train)

print(
    classification_report(
        y_test, balanced_bagging_classifier.predict(scaler.transform(X_test_df_))
    )
)

### Tree-based models

Tree-based models are flexible enough to incorporate minority classes. However, there is a danger of overfitting to these minority observations.

## Modelling subgroups

We don't have to be constrained to using a single model. It may make sense to split our data into subgroups and model each subgroup independently.

We can use a logistic regression model to determine whether our penguins are Gentoos or another species (i.e. not Gentoos).

In [ ]:
gentoo_df = (
    pl.read_csv("data/penguins.csv")
    .with_columns(
        (pl.col("flipper_length_mm") - pl.col("flipper_length_mm").mean())
        / pl.col("flipper_length_mm").std(),
        (pl.col("bill_length_mm") - pl.col("bill_length_mm").mean())
        / pl.col("bill_length_mm").std(),
        pl.when(pl.col("species") == "Gentoo").then(1).otherwise(0).alias("gentoo"),
    )
    .select(["flipper_length_mm", "bill_length_mm", "gentoo"])
    .drop_nulls()
)

gentoo_df.sample(10)

In [ ]:
from sklearn.model_selection import train_test_split

(
    gentoo_feature_train_df,
    gentoo_feature_test_df,
    gentoo_target_train_df,
    gentoo_target_test_df,
) = train_test_split(
    gentoo_df.select(["flipper_length_mm", "bill_length_mm"]),
    gentoo_df.select("gentoo"),
    test_size=0.30,
    random_state=SEED,
)

In [ ]:
logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    gentoo_feature_train_df,
    gentoo_target_train_df,
)

In [ ]:
print(
    classification_report(
        gentoo_target_test_df.get_column("gentoo"),
        logistic_regression_classifier.predict(gentoo_feature_test_df),
    )
)

This very simple model delivers a fairly high degree of accuracy.

We could now build a model to classify the "residuals"---i.e. to distinguish between Adelies and Chinstraps.

What if we use our LDA components with the logistic regression classifier?

In [ ]:
gentoo_lda_df = lda_df.with_columns(
    (pl.col("species") == "Gentoo").alias("gentoo")
).drop("species")

(
    gentoo_lda_feature_train_df,
    gentoo_lda_feature_test_df,
    gentoo_lda_target_train_df,
    gentoo_lda_target_test_df,
) = train_test_split(
    gentoo_lda_df.select(["component1", "component2"]),
    gentoo_lda_df.select("gentoo"),
    test_size=0.30,
    random_state=SEED,
)

lda_logistic_regression_classifier = LogisticRegression(penalty=None)

lda_logistic_regression_classifier.fit(
    gentoo_lda_feature_train_df,
    gentoo_lda_target_train_df,
)

print(
    classification_report(
        gentoo_lda_target_test_df.get_column("gentoo"),
        lda_logistic_regression_classifier.predict(gentoo_lda_feature_test_df),
    )
)

## Hands-on example of selecting features

In this hands-on section you will work with a dataset to perform feature engineering and selection. You can use one of your own datasets, or one from the course. The Lending Club dataset is prepared below, but feel free to substitute your own.

With your chosen dataset, consider the following questions.

- How good is this data. Where did it come from? Can we trust it?
- Look at the distibution of the features?
- Are the target values balanced? If not, what should we do about it?
- What kind of ML algorithm do we want to start with?
- How should we encode the non-numeric features?
- Do we need to scale/transform the data?
- Should we consider using dimensionality reduction?
- Are any of the features highly correlated?
- Are any of the features highly correlated with the target?
- What features is the model prioritising?
- Are there any interaction terms?
- Do we know what features we want to use? How do we choose them? Do we need to search for them?

Feel free to just consider the questions above, rather than write the code to perform the activities.

In [ ]:
def train_validate_test_split(
    X_df, y_df, validate_size=0.25, test_size=0.25, random_state=None, stratify=None
):
    X_remainder_df, X_test_df, y_remainder_df, y_test_df = (
        model_selection.train_test_split(
            X_df,
            y_df,
            test_size=test_size,
            random_state=random_state,
            stratify=stratify,
        )
    )

    X_train_df, X_validate_df, y_train_df, y_validate_df = (
        model_selection.train_test_split(
            X_remainder_df,
            y_remainder_df,
            test_size=validate_size / (1 - test_size),
            random_state=random_state,
            stratify=stratify,
        )
    )

    return X_train_df, X_validate_df, X_test_df, y_train_df, y_validate_df, y_test_df

In [ ]:
lending_club_df = pl.read_parquet("data/lending-club-sample-preprocessed.parquet")

lending_club_df.head()

In [ ]:
(
    lending_club_feature_train_df,
    lending_club_feature_validate_df,
    lending_club_feature_test_df,
    lending_club_target_train_df,
    lending_club_target_validate_df,
    lending_club_target_test_df,
) = train_validate_test_split(
    lending_club_df.drop("fully_paid"),
    lending_club_df.select("fully_paid"),
    validate_size=0.10,
    test_size=0.10,
    random_state=SEED,
)

scaler = StandardScaler()

lc_X_train = scaler.fit_transform(lending_club_feature_train_df)

lc_y_train = lending_club_target_train_df.get_column("fully_paid")

lc_X_validate = scaler.fit_transform(lending_club_feature_validate_df)

lc_y_validate = lending_club_target_validate_df.get_column("fully_paid")

lc_X_test = scaler.fit_transform(lending_club_feature_test_df)

lc_y_test = lending_club_target_test_df.get_column("fully_paid")

In [ ]:
lending_club_classifier = DecisionTreeClassifier(random_state=SEED)

lending_club_classifier.fit(
    lc_X_train,
    lc_y_train,
)

In [ ]:
lending_club_classifier.score(lc_X_validate, lc_y_validate)